In [2]:
import numpy as np 
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
train_dataset = pd.read_csv("./resources/train.csv")
test_dataset = pd.read_csv("./resources/test.csv")

train_label_dataset = train_dataset['SalePrice'] 

In [4]:
train_dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [275]:
train_dataset[['SalePrice']].describe().T

,count,mean,std,min,25%,50%,75%,max
SalePrice,1460.0,180921.19589,79442.502883,34900.0,129975.0,163000.0,214000.0,755000.0


In [276]:
remove_columns = set()
remove_columns.add('SalePrice')

In [277]:
def sale_price_by_column(column: str, is_remove: bool = False):
    if is_remove:
        remove_columns.add(column)
    return train_dataset[[column,'SalePrice']].groupby([column]).describe()

In [278]:
# 차도 유형 - 포장 or 비포장
# 특성 제거 : Grvl 이 6개밖에 없어서 비율이 현저히 적다고 판단 
sale_price_by_column('Street', True)

SalePrice                                                            \
           count           mean           std      min       25%       50%   
Street                                                                       
Grvl         6.0  130190.500000  65446.253991  55993.0   88250.0  114250.0   
Pave      1454.0  181130.538514  79446.597317  34900.0  130000.0  163000.0   

                            
             75%       max  
Street                      
Grvl    169650.0  228950.0  
Pave    214000.0  755000.0

In [279]:
# 골목길 유형 - 포장 or 비포장
sale_price_by_column('Alley')

SalePrice                                                            \
          count           mean           std      min       25%       50%   
Alley                                                                       
Grvl       50.0  122219.080000  34780.780734  52500.0  105312.5  119500.0   
Pave       41.0  168000.585366  38370.375243  40000.0  151000.0  172500.0   

                           
            75%       max  
Alley                      
Grvl   136750.0  256000.0  
Pave   185000.0  265979.0

In [280]:
# LotShape
# 특성 제거 : IR1 min이 52000이고, max가 75만이기 때문에 해당 특성이 price에 영향을 준다는 신뢰가 낮다고 판단
sale_price_by_column('LotShape', True)

SalePrice                                                             \
             count           mean           std       min       25%       50%   
LotShape                                                                        
IR1          484.0  206101.665289  85858.489559   52000.0  150000.0  189000.0   
IR2           41.0  239833.365854  99669.427362  110000.0  175000.0  221000.0   
IR3           10.0  216036.500000  82540.334855   73000.0  167875.0  203570.0   
Reg          925.0  164754.818378  69673.427215   34900.0  120000.0  146000.0   

                              
               75%       max  
LotShape                      
IR1       239000.0  755000.0  
IR2       250000.0  538000.0  
IR3       265000.0  375000.0  
Reg       188000.0  582933.0

In [281]:
# LandContour - 평탄함 정도
sale_price_by_column('LandContour')

SalePrice                                                   \
                count           mean            std      min       25%   
LandContour                                                              
Bnk              63.0  143104.079365   49361.244074  52500.0  113000.0   
HLS              50.0  231533.940000  101790.139741  82500.0  151750.0   
Low              36.0  203661.111111   83935.353620  39300.0  143000.0   
Lvl            1311.0  180183.746758   78463.567918  34900.0  130000.0   

                                            
                  50%        75%       max  
LandContour                                 
Bnk          139400.0  171250.00  315000.0  
HLS          222250.0  281347.25  538000.0  
Low          190000.0  263750.00  385000.0  
Lvl          162900.0  212000.00  755000.0

In [282]:
# Utilities 
# 특성 제거 : NoSeWa 카운트가 1밖에 안돼서 너무 적다고 판단
sale_price_by_column('Utilities', True)

SalePrice                                                            \
              count          mean           std       min       25%       50%   
Utilities                                                                       
AllPub       1459.0  180950.95682  79461.599814   34900.0  129950.0  163000.0   
NoSeWa          1.0  137500.00000           NaN  137500.0  137500.0  137500.0   

                               
                75%       max  
Utilities                      
AllPub     214000.0  755000.0  
NoSeWa     137500.0  137500.0

In [283]:
# LotConfig 
# 특성 제거 : min과 Max 차이 
sale_price_by_column('LotConfig', True)

SalePrice                                                   \
              count           mean           std       min       25%   
LotConfig                                                              
Corner        263.0  181623.425856  84466.041222   52500.0  129250.0   
CulDSac        94.0  223854.617021  93117.546586   84000.0  156475.0   
FR2            47.0  177934.574468  62788.926829   81000.0  143500.0   
FR3             4.0  208475.000000  78379.222374  128000.0  169925.0   
Inside       1052.0  176938.047529  76426.805787   34900.0  128000.0   

                                         
                50%       75%       max  
LotConfig                                
Corner     160000.0  220000.0  755000.0  
CulDSac    199262.0  269342.5  625000.0  
FR2        165000.0  194500.0  394617.0  
FR3        195450.0  234000.0  315000.0  
Inside     159697.5  207125.0  611657.0

In [284]:
# LandSlope - 경사도
# 특성 제거 : min과 Max 차이 
sale_price_by_column('LandSlope', True)

SalePrice                                                            \
              count           mean           std      min       25%       50%   
LandSlope                                                                       
Gtl          1382.0  179956.799566  78669.369151  34900.0  129900.0  161875.0   
Mod            65.0  196734.138462  92375.358116  39300.0  130000.0  186700.0   
Sev            13.0  204379.230769  86729.922409  61000.0  143000.0  185000.0   

                               
                75%       max  
LandSlope                      
Gtl        212000.0  755000.0  
Mod        259500.0  538000.0  
Sev        260000.0  375000.0

In [285]:
# Neighborhood - 중간값 확인해보면 max 가 높은 순으로 정렬돼있음 
sale_price_by_column('Neighborhood')

SalePrice                                                     \
                 count           mean            std       min        25%   
Neighborhood                                                                
Blmngtn           17.0  194870.882353   30393.229219  159895.0  174000.00   
Blueste            2.0  137500.000000   19091.883092  124000.0  130750.00   
BrDale            16.0  104493.750000   14330.176493   83000.0   91000.00   
BrkSide           58.0  124834.051724   40348.689270   39300.0  100500.00   
ClearCr           28.0  212565.428571   50231.538993  130000.0  183750.00   
CollgCr          150.0  197965.773333   51403.666438  110000.0  152958.75   
Crawfor           51.0  210624.725490   68866.395472   90350.0  159250.00   
Edwards          100.0  128219.700000   43208.616459   58500.0  101500.00   
Gilbert           79.0  192854.506329   35986.779085  141000.0  174000.00   
IDOTRR            37.0  100123.783784   33376.710117   34900.0   81000.00   
MeadowV           17.0   98576.470588   23491.049610   75000.0   83500.00   
Mitchel           49.0  156270.122449   36486.625334   84500.0  131000.00   
NAmes            225.0  145847.080000   33075.345450   87500.0  127500.00   
NPkVill            9.0  142694.444444    9377.314529  127500.0  140000.00   
NWAmes            73.0  189050.068493   37172.218106   82500.0  165150.00   
NoRidge           41.0  335295.317073  121412.658640  190000.0  265000.00   
NridgHt           77.0  316270.623377   96392.544954  154000.0  253293.00   
OldTown          113.0  128225.300885   52650.583185   37900.0  105900.00   
SWISU             25.0  142591.360000   32622.917679   60000.0  128000.00   
Sawyer            74.0  136793.135135   22345.129157   62383.0  127250.00   
SawyerW           59.0  186555.796610   55651.997820   76000.0  145500.00   
Somerst           86.0  225379.837209   56177.555888  144152.0  177975.00   
StoneBr           25.0  310499.000000  112969.676640  170000.0  213500.00   
Timber            38.0  242247.447368   64845.651549  137500.0  186900.00   
Veenker           11.0  238772.727273   72369.317959  162500.0  184250.00   

                                             
                   50%        75%       max  
Neighborhood                                 
Blmngtn       191000.0  213490.00  264561.0  
Blueste       137500.0  144250.00  151000.0  
BrDale        106000.0  118000.00  125000.0  
BrkSide       124300.0  141175.00  223500.0  
ClearCr       200250.0  242225.00  328000.0  
CollgCr       197200.0  225725.00  424870.0  
Crawfor       200624.0  239000.00  392500.0  
Edwards       121750.0  145225.00  320000.0  
Gilbert       181000.0  197200.00  377500.0  
IDOTRR        103000.0  120500.00  169500.0  
MeadowV        88000.0  115000.00  151400.0  
Mitchel       153500.0  171000.00  271000.0  
NAmes         140000.0  158000.00  345000.0  
NPkVill       146000.0  148500.00  155000.0  
NWAmes        182900.0  205000.00  299800.0  
NoRidge       301500.0  341000.00  755000.0  
NridgHt       315000.0  374000.00  611657.0  
OldTown       119000.0  140000.00  475000.0  
SWISU         139500.0  160000.00  200000.0  
Sawyer        135000.0  149462.50  190000.0  
SawyerW       179900.0  222500.00  320000.0  
Somerst       225500.0  252919.50  423000.0  
StoneBr       278000.0  377426.00  556581.0  
Timber        228475.0  286115.75  378500.0  
Veenker       218000.0  282000.00  385000.0

In [286]:
# Condition1 
# 특성 제거 : 중간값 및 max 값 랭크가 불일치
sale_price_by_column('Condition1', True)

SalePrice                                                    \
               count           mean           std       min        25%   
Condition1                                                               
Artery          48.0  135091.666667  66226.660548   66500.0  105000.00   
Feedr           81.0  142475.481481  42157.290117   40000.0  121600.00   
Norm          1260.0  184495.492063  81256.980125   34900.0  131500.00   
PosA             8.0  225875.000000  52348.932313  180000.0  188750.00   
PosN            19.0  215184.210526  65256.536609  109500.0  167250.00   
RRAe            11.0  138400.000000  24030.813553   87000.0  127750.00   
RRAn            26.0  184396.615385  66177.922070   79500.0  152393.75   
RRNe             2.0  190750.000000   5303.300859  187000.0  188875.00   
RRNn             5.0  212400.000000  93823.238060  110000.0  128000.00   

                                          
                 50%       75%       max  
Condition1                                
Artery      119550.0  143000.0  475000.0  
Feedr       140000.0  167500.0  244600.0  
Norm        166500.0  219500.0  755000.0  
PosA        212500.0  244000.0  335000.0  
PosN        200000.0  252250.0  385000.0  
RRAe        142500.0  156500.0  171000.0  
RRAn        171495.0  190105.0  423000.0  
RRNe        190750.0  192625.0  194500.0  
RRNn        214000.0  290000.0  320000.0

In [287]:
# Condition2 
# 특성 제거 : Norm 제외 개수가 너무 없음 
sale_price_by_column('Condition2', True)

SalePrice                                                    \
               count           mean            std       min       25%   
Condition2                                                               
Artery           2.0  106500.000000   16263.455967   95000.0  100750.0   
Feedr            6.0  121166.666667   33544.994659   79500.0   95500.0   
Norm          1445.0  181169.405536   79337.735547   34900.0  130000.0   
PosA             1.0  325000.000000            NaN  325000.0  325000.0   
PosN             2.0  284875.000000  141598.132933  184750.0  234812.5   
RRAe             1.0  190000.000000            NaN  190000.0  190000.0   
RRAn             1.0  136905.000000            NaN  136905.0  136905.0   
RRNn             2.0   96750.000000   39951.533137   68500.0   82625.0   

                                          
                 50%       75%       max  
Condition2                                
Artery      106500.0  112250.0  118000.0  
Feedr       127500.0  137000.0  167500.0  
Norm        163500.0  214000.0  755000.0  
PosA        325000.0  325000.0  325000.0  
PosN        284875.0  334937.5  385000.0  
RRAe        190000.0  190000.0  190000.0  
RRAn        136905.0  136905.0  136905.0  
RRNn         96750.0  110875.0  125000.0

In [288]:
# BldgType -
sale_price_by_column('BldgType')

SalePrice                                                            \
             count           mean           std      min       25%       50%   
BldgType                                                                       
1Fam        1220.0  185763.807377  82648.502922  34900.0  131475.0  167900.0   
2fmCon        31.0  128432.258065  35458.545158  55000.0  106875.0  127500.0   
Duplex        52.0  133541.076923  27833.249197  82000.0  118375.0  135980.0   
Twnhs         43.0  135911.627907  41013.222080  75000.0   95750.0  137500.0   
TwnhsE       114.0  181959.342105  60626.108918  75500.0  143187.5  172200.0   

                              
               75%       max  
BldgType                      
1Fam      222000.0  755000.0  
2fmCon    142500.0  228950.0  
Duplex    145000.0  206300.0  
Twnhs     168750.0  230000.0  
TwnhsE    207375.0  392500.0

In [289]:
# HouseStyle - 
sale_price_by_column('HouseStyle')

SalePrice                                                    \
               count           mean            std       min       25%   
HouseStyle                                                               
1.5Fin         154.0  143116.740260   54277.941119   37900.0  114625.0   
1.5Unf          14.0  110150.000000   19036.473090   76000.0   98175.0   
1Story         726.0  175985.477961   77055.715011   34900.0  127000.0   
2.5Fin           8.0  220000.000000  118211.976671  104000.0  164250.0   
2.5Unf          11.0  157354.545455   63934.128032  101000.0  125000.0   
2Story         445.0  210051.764045   87339.213570   40000.0  159500.0   
SFoyer          37.0  135074.486486   30480.898192   75500.0  127500.0   
SLvl            65.0  166703.384615   38305.161339   91000.0  145000.0   

                                          
                 50%       75%       max  
HouseStyle                                
1.5Fin      132000.0  159325.5  410000.0  
1.5Unf      111250.0  120500.0  139400.0  
1Story      154750.0  209350.0  611657.0  
2.5Fin      194000.0  223750.0  475000.0  
2.5Unf      133900.0  163500.0  325000.0  
2Story      190000.0  240000.0  755000.0  
SFoyer      135960.0  148000.0  206300.0  
SLvl        164500.0  178000.0  345000.0

In [290]:
# RoofStyle
# 특성 제거 : min과 Max 차이 
sale_price_by_column('RoofStyle', True)

SalePrice                                                    \
              count           mean            std       min       25%   
RoofStyle                                                               
Flat           13.0  194690.000000   62522.963515   82000.0  143000.0   
Gable        1141.0  171483.956179   66331.237296   34900.0  128000.0   
Gambrel        11.0  148909.090909   67013.527807   40000.0  105500.0   
Hip           286.0  218876.933566  111549.603563   55000.0  139675.0   
Mansard         7.0  180568.428571   58057.624627  100000.0  145500.0   
Shed            2.0  225000.000000   49497.474683  190000.0  207500.0   

                                         
                50%       75%       max  
RoofStyle                                
Flat       185000.0  242000.0  274970.0  
Gable      160000.0  202500.0  755000.0  
Gambrel    139000.0  193750.0  259500.0  
Hip        176500.0  277875.0  745000.0  
Mansard    175000.0  216000.0  265979.0  
Shed       225000.0  242500.0  260000.0

In [291]:
# RoofMatl
# 특성 제거 : WdShngl 제외 개수가 너무 없음 
sale_price_by_column('RoofMatl', True)

SalePrice                                                    \
             count           mean            std       min       25%   
RoofMatl                                                               
ClyTile        1.0  160000.000000            NaN  160000.0  160000.0   
CompShg     1434.0  179803.679219   77722.388636   34900.0  129900.0   
Membran        1.0  241500.000000            NaN  241500.0  241500.0   
Metal          1.0  180000.000000            NaN  180000.0  180000.0   
Roll           1.0  137000.000000            NaN  137000.0  137000.0   
Tar&Grv       11.0  185406.363636   65430.141720   82000.0  136000.0   
WdShake        5.0  241400.000000   36218.779659  190000.0  228000.0   
WdShngl        6.0  390250.000000  206969.019421  168500.0  278500.0   

                                        
               50%       75%       max  
RoofMatl                                
ClyTile   160000.0  160000.0  160000.0  
CompShg   162000.0  213000.0  745000.0  
Membran   241500.0  241500.0  241500.0  
Metal     180000.0  180000.0  180000.0  
Roll      137000.0  137000.0  137000.0  
Tar&Grv   167000.0  249000.0  274970.0  
WdShake   242000.0  260000.0  287000.0  
WdShngl   332500.0  452500.0  755000.0

In [292]:
# Exterior1st - 중간값 비슷
sale_price_by_column('Exterior1st')

SalePrice                                                    \
                count           mean            std       min       25%   
Exterior1st                                                               
AsbShng          20.0  107385.550000   33756.480660   35311.0   85750.0   
AsphShn           1.0  100000.000000            NaN  100000.0  100000.0   
BrkComm           2.0   71000.000000   15556.349186   60000.0   65500.0   
BrkFace          50.0  194573.000000   82841.914370   40000.0  134525.0   
CBlock            1.0  105000.000000            NaN  105000.0  105000.0   
CemntBd          61.0  231690.655738  120575.621318   75000.0  119500.0   
HdBoard         222.0  163077.450450   66305.714164   83000.0  129625.0   
ImStucc           1.0  262000.000000            NaN  262000.0  262000.0   
MetalSd         220.0  149422.177273   54776.408990   62383.0  117750.0   
Plywood         108.0  175942.379630   49497.383293   82500.0  143437.5   
Stone             2.0  258500.000000   40305.086528  230000.0  244250.0   
Stucco           25.0  162990.000000   83307.246529   58500.0  116000.0   
VinylSd         515.0  213732.900971   80646.975785   37900.0  160000.0   
Wd Sdng         206.0  149841.645631   71130.448152   34900.0  112625.0   
WdShing          26.0  150655.076923   72507.530741   87000.0  109750.0   

                                           
                  50%       75%       max  
Exterior1st                                
AsbShng      108000.0  133500.0  165500.0  
AsphShn      100000.0  100000.0  100000.0  
BrkComm       71000.0   76500.0   82000.0  
BrkFace      165750.0  245125.0  430000.0  
CBlock       105000.0  105000.0  105000.0  
CemntBd      236500.0  303477.0  556581.0  
HdBoard      149900.0  179900.0  755000.0  
ImStucc      262000.0  262000.0  262000.0  
MetalSd      139000.0  164775.0  392000.0  
Plywood      167450.0  197500.0  345000.0  
Stone        258500.0  272750.0  287000.0  
Stucco       144000.0  188700.0  381000.0  
VinylSd      200000.0  248614.0  611657.0  
Wd Sdng      138943.5  167250.0  745000.0  
WdShing      128700.0  160250.0  385000.0

In [293]:
# Exterior2nd
# 특성 제거 : 카운트가 비교적 적은 것이 영향을 많이 주고, 표준편차 큼
sale_price_by_column('Exterior2nd', True)

SalePrice                                                    \
                count           mean            std       min       25%   
Exterior2nd                                                               
AsbShng          20.0  114060.550000   42315.043608   35311.0   94000.0   
AsphShn           3.0  138000.000000   37509.998667  100000.0  119500.0   
Brk Cmn           7.0  126714.285714   38693.084161   60000.0  114000.0   
BrkFace          25.0  195818.000000   95097.551756   40000.0  137000.0   
CBlock            1.0  105000.000000            NaN  105000.0  105000.0   
CmentBd          60.0  230093.833333  116140.396221   75000.0  118375.0   
HdBoard         207.0  167661.565217   70061.078357   83000.0  130500.0   
ImStucc          10.0  252070.000000  193176.672447   88000.0  131250.0   
MetalSd         214.0  149803.172897   55078.595310   62383.0  118625.0   
Other             1.0  319000.000000            NaN  319000.0  319000.0   
Plywood         142.0  168112.387324   46955.744372   76500.0  135000.0   
Stone             5.0  158224.800000   63533.067573   90000.0   93500.0   
Stucco           26.0  155905.153846   74862.492410   58500.0   92262.5   
VinylSd         504.0  214432.460317   80708.309670   37900.0  161550.0   
Wd Sdng         197.0  148386.065990   59892.856640   34900.0  114500.0   
Wd Shng          38.0  161328.947368   88974.217820   55000.0  117000.0   

                                            
                  50%        75%       max  
Exterior2nd                                 
AsbShng      111000.0  135500.00  225000.0  
AsphShn      139000.0  157000.00  175000.0  
Brk Cmn      147000.0  148500.00  155000.0  
BrkFace      160000.0  250000.00  430000.0  
CBlock       105000.0  105000.00  105000.0  
CmentBd      238750.0  304082.75  556581.0  
HdBoard      155000.0  182500.00  755000.0  
ImStucc      187600.0  305500.00  745000.0  
MetalSd      138750.0  164525.00  392000.0  
Other        319000.0  319000.00  319000.0  
Plywood      160750.0  192750.00  320000.0  
Stone        177000.0  200624.00  230000.0  
Stucco       142000.0  197250.00  381000.0  
VinylSd      200070.5  249100.00  611657.0  
Wd Sdng      138000.0  164900.00  475000.0  
Wd Shng      138225.0  172312.50  538000.0

In [294]:
# MasVnrType
sale_price_by_column('MasVnrType')

SalePrice                                                   \
               count           mean           std       min       25%   
MasVnrType                                                              
BrkCmn          15.0  146318.066667  46187.583632   89471.0  114250.0   
BrkFace        445.0  204691.871910  81214.293554   75000.0  149300.0   
Stone          128.0  265583.625000  99940.156577  119000.0  194650.0   

                                          
                 50%       75%       max  
MasVnrType                                
BrkCmn      139000.0  163950.0  277000.0  
BrkFace     181000.0  236000.0  755000.0  
Stone       246839.0  312779.0  611657.0

In [295]:
# ExterQual - max, min , 중간값 명확함
sale_price_by_column('ExterQual')

SalePrice                                                    \
              count           mean            std       min       25%   
ExterQual                                                               
Ex             52.0  367360.961538  116401.264200  160000.0  311404.0   
Fa             14.0   87985.214286   39826.918794   39300.0   60250.0   
Gd            488.0  231633.510246   71188.873899   52000.0  185000.0   
TA            906.0  144341.313466   42471.815703   34900.0  118589.5   

                                          
                50%        75%       max  
ExterQual                                 
Ex         364606.5  428788.50  755000.0  
Fa          82250.0  102000.00  200000.0  
Gd         220000.0  265984.25  745000.0  
TA         139450.0  165500.00  381000.0

In [296]:
# ExterCond
sale_price_by_column('ExterCond')

SalePrice                                                    \
              count           mean            std       min       25%   
ExterCond                                                               
Ex              3.0  201333.333333  109235.220205  118000.0  139500.0   
Fa             28.0  102595.142857   40094.383940   39300.0   65500.0   
Gd            146.0  168897.568493   72608.303632   68400.0  128625.0   
Po              1.0   76500.000000            NaN   76500.0   76500.0   
TA           1282.0  184034.896256   79806.257233   34900.0  131100.0   

                                          
                50%        75%       max  
ExterCond                                 
Ex         161000.0  243000.00  325000.0  
Fa          95750.0  137750.00  169500.0  
Gd         151250.0  187375.00  625000.0  
Po          76500.0   76500.00   76500.0  
TA         167370.0  217334.25  755000.0

In [297]:
from sklearn.model_selection import train_test_split

##########데이터 로드

x_data = np.array([
    [2, 1],
    [3, 2],
    [3, 4],
    [5, 5],
    [7, 5],
    [2, 5],
    [8, 9],
    [9, 10],
    [6, 12],
    [9, 2],
    [6, 10],
    [2, 4]
])
y_data = np.array([3, 5, 7, 10, 12, 7, 13, 13, 12, 13, 12, 6])

##########데이터 분석

##########데이터 전처리

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777)

##########모델 생성

model = GradientBoostingRegressor()

##########모델 학습

model.fit(x_train, y_train)

##########모델 검증

print(model.score(x_train, y_train)) #

print(model.score(x_test, y_test)) #0.7421680021828538

##########모델 예측

x_test = np.array([
    [4, 6]
])

y_predict = model.predict(x_test)

print(y_predict[0]) #6.497766320678856

0.9999999992944921
0.7309323996910557
6.449858454393457


### Encoding

In [314]:
from sklearn.preprocessing import LabelEncoder


label_encoder= LabelEncoder()
all_dataset = train_dataset._append(test_dataset)

for column in all_dataset.columns:
    if(column == 'Id'):
        continue
    encoded = label_encoder.fit_transform(all_dataset[column])
    all_dataset[column] = encoded
    

In [315]:
train_length = 1460
encoded_train_dataset = all_dataset.iloc[:train_length]
encoded_test_dataset = all_dataset.iloc[train_length:]

In [316]:
filtered_train_dataset = encoded_train_dataset.drop(columns=list(remove_columns))
filtered_test_dataset = encoded_test_dataset.drop(columns=list(remove_columns))

In [317]:
encoded_test_dataset

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,0,2,56,1329,1,2,3,3,0,...,0,3,2,4,0,5,4,8,4,663
1,1462,0,3,57,1671,1,2,0,3,0,...,0,3,4,0,35,5,4,8,4,663
2,1463,5,3,50,1635,1,2,0,3,0,...,0,3,2,4,0,2,4,8,4,663
3,1464,5,3,54,979,1,2,0,3,0,...,0,3,4,4,0,5,4,8,4,663
4,1465,11,3,19,178,1,2,0,1,0,...,0,3,4,4,0,0,4,8,4,663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,13,4,0,23,1,2,3,3,0,...,0,3,4,4,0,5,0,8,4,663
1455,2916,13,4,0,20,1,2,3,3,0,...,0,3,4,4,0,3,0,8,0,663
1456,2917,0,3,121,1863,1,2,3,3,0,...,0,3,4,4,0,8,0,8,0,663
1457,2918,9,3,38,1090,1,2,3,3,0,...,0,3,2,2,17,6,0,8,4,663


### 결측값 채우기

In [318]:
filtered_train_dataset.fillna('')
encoded_test_dataset.fillna('')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,0,2,56,1329,1,2,3,3,0,...,0,3,2,4,0,5,4,8,4,663
1,1462,0,3,57,1671,1,2,0,3,0,...,0,3,4,0,35,5,4,8,4,663
2,1463,5,3,50,1635,1,2,0,3,0,...,0,3,2,4,0,2,4,8,4,663
3,1464,5,3,54,979,1,2,0,3,0,...,0,3,4,4,0,5,4,8,4,663
4,1465,11,3,19,178,1,2,0,1,0,...,0,3,4,4,0,0,4,8,4,663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,13,4,0,23,1,2,3,3,0,...,0,3,4,4,0,5,0,8,4,663
1455,2916,13,4,0,20,1,2,3,3,0,...,0,3,4,4,0,3,0,8,0,663
1456,2917,0,3,121,1863,1,2,3,3,0,...,0,3,4,4,0,8,0,8,0,663
1457,2918,9,3,38,1090,1,2,3,3,0,...,0,3,2,2,17,6,0,8,4,663


In [319]:
model = GradientBoostingRegressor()

model.fit(filtered_train_dataset, train_label_dataset)
# print(model.score(x_train, y_train))

GradientBoostingRegressor()

In [320]:
model.predict(filtered_test_dataset)

array([126112.61035801, 162544.82095622, 180373.09477691, ...,
       169584.50897565, 111628.36749524, 241594.90305487])